# Folder-wise Fast Microscopy Picture Analysis
The following code opens a folder of the Visitron Microscope and automatically sets ROIs to the darkest and brightest portions of the picture. Consequently it removes the background through the ROIs at the darkest positions and calculates the mean fluorescence at the brighter spots. 

In [ ]:
## Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import tifffile as tiff
import tkinter as tk
import imagecodecs
from tkinter.filedialog import askopenfilename,askdirectory
import os
from skimage import filters, measure, morphology, draw, exposure
import math
import pandas as pd
from PIL import Image
from PIL import Image
from lxml import etree

In [ ]:
# Use tkinter to open a file dialog and select the folder with images
def select_folder():
    root = tk.Tk()
    root.withdraw()  # We don't want a full GUI, so keep the root window from appearing
    folder_path = askdirectory(title="Select Folder with Channel TIFF Images")
    root.destroy()
    return folder_path

# Function Definitions

In [ ]:
# Normalize images for display
def normalize(image):
    return (image - np.min(image)) / (np.max(image) - np.min(image))

In [ ]:
def read_tiff_channel(file_path):
    """Read a TIFF channel image."""
    return tiff.imread(file_path)

def read_ome_file(ome_path):
    """Parse the .ome file and return the metadata."""
    with open(ome_path, 'r') as file:
        ome_content = file.read()
    ome_xml = etree.fromstring(ome_content)
    return ome_xml

In [ ]:
def process_image_folder(folder_path, return_metadata=False, return_comb_images = True):
    """Process all images and their channels in the specified folder.
    Returns a list of combined images."""
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.ome'):
                # Use the .ome file name as the base name for the images
                base_name = os.path.splitext(file)[0]
                ome_path = os.path.join(root, file)
                if return_metadata: ome_metadata = read_ome_file(ome_path)

                # Find the associated channel images with variable suffixes
                channel_files = [f for f in files if f.startswith(base_name) and f.endswith(('.tiff', '.tf2'))]
                channel_files.sort()  # Sort files alphabetically

                channels = []
                for channel_file in channel_files:
                    channel_path = os.path.join(root, channel_file)
                    channel_image = read_tiff_channel(channel_path)
                    channels.append(channel_image)

                # Combine the channels into a single image
                if return_comb_images:    
                    if len(channels) == 3:
                        combined_image = Image.merge("RGB", [Image.fromarray(channel) for channel in channels])
                    elif len(channels) == 4:
                        combined_image = Image.merge("RGBA", [Image.fromarray(channel) for channel in channels])
                    else:
                        print(f"Unexpected number of channels ({len(channels)}) for image {base_name}")
                        continue

    return channels,combined_image, ome_metadata if return_metadata and return_comb_images else channels, ome_metadata if return_metadata else channels,combined_image if return_comb_images else channels

In [ ]:
def get_images(folder_path):
    # Get all files in the folder
    files = os.listdir(folder_path)
    # Get only tiff files
    files = [f for f in files if f.endswith('.tif') or f.endswith('.tiff')or f.endswith('.tf2')]
    # Sort files by channel
    files.sort()
    # Initialize empty list to store images
    images = []
    # Loop through all files
    for file in files:
        # Read image
        image = tiff.imread(folder_path + '/' + file)
        # Append image to list
        images.append(image)
    return images

In [ ]:
def background_ROI_substraction(image, background_threshold = 0):
    n_channels = image.shape[2]

    # Initialize empty list to store background values
    background_values = {f'Channel {i+1}': [] for i in range(n_channels)}
    
    # Initialize empty image to store background subtracted image
    background_subtracted_image = np.copy(image)

    background_threshold_temp = background_threshold
    # fig, axs = plt.subplots(1, n_channels, figsize=(n_channels*4,18))

    # for ax, channel_index in zip(axs, range(n_channels)):
    #     ax.imshow(image[:, :, channel_index], cmap='gray')
    #     ax.set_title(f'Channel {channel_index+1}')
    
    for channel_index in range(n_channels):
        channel = image[:, :, channel_index]
        # Define background threshold, by standard deviation from the mean, if not provided
        if background_threshold_temp == 0:
            background_threshold = 2 * np.std(channel)
        background_thresh = channel < background_threshold
        print(f'Background threshold for channel {channel_index+1}: {background_threshold}')

        # Label the background regions
        background_labels = measure.label(background_thresh)
        background_props = measure.regionprops(background_labels)

        # Create bounding boxes around the detected background contours to define background ROIs
        background_rois = []

        for prop in background_props:
            minr, minc, maxr, maxc = prop.bbox[:4]
            background_rois.append((minc, minr, maxc - minc, maxr - minr))

        # Calculate mean background values for the channel
        for roi in background_rois:
            x, y, w, h = roi
            roi_area = channel[y:y+h, x:x+w]
            mean_background = np.mean(roi_area)
            background_values[f'Channel {channel_index+1}'].append(mean_background)
        
        # Display background ROIs on the current channel
        roi_image = np.stack([normalize(channel)]*3, axis=-1)  # Convert to RGB
        for roi in background_rois:
            x, y, w, h = roi
            rr, cc = draw.rectangle_perimeter((y, x), extent=(h, w), shape=channel.shape)
            roi_image[rr, cc] = [1, 0, 0]  # Red for ROIs

        plt.imshow(roi_image)
        plt.title(f'Background ROIs on Channel {channel_index + 1}')
        plt.show()

    #     for ax in axs:
    #         for roi in background_rois:
    #             x, y, w, h = roi
    #             rect = plt.Rectangle((x, y), w, h, edgecolor='r', facecolor='none')
    #             ax.add_patch(rect)
    #     # print(f'Number of background ROIs for channel {channel_index+1}: {len(background_rois)}')
    #     # print(f'Mean background values for channel {channel_index+1}: {background_values[f"Channel {channel_index+1}"]}')
    
    # plt.show()
    
    # Subtract background from the entire image
    mean_background_value = []
    for channel_index in range(n_channels):
        mean_background_value.append(np.mean(background_values[f'Channel {channel_index+1}']))
        print(f'Mean background value for channel {channel_index+1}: {mean_background_value[channel_index]}')
        background_subtracted_image[:, :, channel_index] -= np.minimum(math.floor(mean_background_value[channel_index]), background_subtracted_image[:, :, channel_index])

    return background_values, mean_background_value, background_subtracted_image

In [ ]:
def background_ROI_substraction_single_ch(image, background_threshold = 0,channel_of_interest = 0):
    
    n_channels = image.shape[2]

    # Initialize empty list to store background values
    background_values = {f'Channel {i+1}': [] for i in range(n_channels)}
    
    # Initialize empty image to store background subtracted image
    bg_subs_image_single_ch = image[:,:,channel_of_interest]

    # Define background threshold, by standard deviation from the mean, if not provided
    if background_threshold == 0:
            background_threshold = np.mean(bg_subs_image_single_ch) - 0.2 * np.std(bg_subs_image_single_ch)
            if background_threshold < 0:
                background_threshold = np.min(bg_subs_image_single_ch) + 0.01 * np.std(bg_subs_image_single_ch)
            print(f'Background threshold for channel {channel_of_interest+1}: {background_threshold}')
    
    # Find minima below a threshold to define background ROIs

    background_thresh = bg_subs_image_single_ch < background_threshold

    # Label the background regions
    background_labels = measure.label(background_thresh)
    background_props = measure.regionprops(background_labels)

    # Create bounding boxes around the detected background contours to define background ROIs
    background_rois = []

    for prop in background_props:
        minr, minc, maxr, maxc = prop.bbox[:4]
        background_rois.append((minc, minr, maxc - minc, maxr - minr))

    # Display background ROIs on channel of interest
    background_roi_image = np.stack([normalize(bg_subs_image_single_ch)]*3, axis=-1)  # Convert to RGB

    for roi in background_rois:
        x, y, w, h = roi
        background_roi_image[y:y+h, x:x+w] = [1, 0, 0]  # Red for background ROIs

    plt.imshow(background_roi_image)
    plt.title(f'Background ROIs on Channel of Interest {channel_of_interest + 1}')
    plt.show()

    # Calculate mean background values for each channel
    for channel_index in range(n_channels):
        channel = image[:, :, channel_index]
        
        for roi in background_rois:
            x, y, w, h = roi
            roi_area = channel[y:y+h, x:x+w]
            mean_background = np.mean(roi_area)
            background_values[f'Channel {channel_index+1}'].append(mean_background)

    mean_background_value = []
    # Subtract background from the entire image
    background_subtracted_image = np.copy(image)
    
    for channel_index in range(n_channels):
        mean_background_value.append(np.nan_to_num(np.mean(background_values[f'Channel {channel_index+1}'])))
        background_subtracted_image[:, :, channel_index] -= np.minimum(math.floor(mean_background_value[channel_index]), background_subtracted_image[:, :, channel_index])
    
    return background_values, mean_background_value, background_subtracted_image

In [ ]:
def process_images(channels, lower_thresh_chan, upper_thresh, background_threshold = 0, radius_factor = 10, mask_channel = 1, channel_of_interest = 1, single_ch_background = True):
    '''
    # Function to process a single set of 4-channel images and return mean fluorescence values, mean background values, and number of positive and negative results

            Parameters:
                base_name (string): The path to the folder containing the 4-channel images
                lower_thresh_chan (int): A list of 4 integers, the lower threshold for each channel
                upper_thresh (int): The upper threshold for the channel of interest
                background_threshold (int): The threshold for the background values, if left empty, it will be 2 standard deviations below the mean
                radius_factor (int): The radius of the circular ROIs around detected points
                channel_of_interest (int): The channel of interest, default is 1

            Returns:
                mean_fluorescence (dict): A dictionary containing the mean fluorescence values for each channel
                mean_fluorescence_value (float): The mean fluorescence value for each channel
                background_values (dict): A dictionary containing the mean background values for each channel
                mean_background_value (float): The mean background value for the channel of interest
                positive_results (dict): A dictionary containing the number of positive results for each channel
                negative_results (dict): A dictionary containing the number of negative results for each channel
    '''
    channel_of_interest -= 1
    mask_channel -= 1

    channel1_raw = channels[0]
    channel2_raw = channels[1]
    channel3_raw = channels[2]
    if len(channels) == 4:
        channel4_raw = channels[3]
    
    
    channel1 = channel1_raw[:, :] # Blue Channel  
    channel2 = channel2_raw[:, :] # Green Channel
    channel3 = channel3_raw[:, :] # Red Channel
    if len(channels) == 4:
        channel4 = channel4_raw[:, :] # Far Red Channel

    # Combine channels into a single multi-channel image for convenience
    if len(channels) == 4:
        image = np.stack((channel1, channel2, channel3, channel4), axis=-1)
    else:
        image = np.stack((channel1, channel2, channel3), axis=-1) 

    n_channels = image.shape[2]

    # Initialize dictonaries to store the mean fluorescence and background values 
    mean_fluorescence = {f'Channel {i+1}': [] for i in range(n_channels)}
    background_values = {f'Channel {i+1}': [] for i in range(n_channels)}
    positive_results = {f'Channel {i+1}': [] for i in range(n_channels)}

    # Subtract background from the image and display the background ROIs on the channel of interest
    if single_ch_background:
        background_values, mean_background_value, background_subtracted_image = background_ROI_substraction_single_ch(image, background_threshold,channel_of_interest)
    else:
        background_values, mean_background_value, background_subtracted_image = background_ROI_substraction(image, background_threshold)
    
    fig, axs = plt.subplots(1, n_channels , figsize=(n_channels*4, 10))

    for ax, channel_index in zip(axs, range(n_channels)):
        ax.hist(image[:, :, channel_index].ravel(), bins=256, color='gray', alpha=0.75)
        ax.set_title(f"Histogram for Channel {channel_index+1}")
        ax.set_xlabel("Pixel intensity")
        ax.set_ylabel("Frequency")
        ax.set_yscale('log')
        ax.set_xscale('log')
        ax.axvline(mean_background_value[channel_index], color='r', linestyle='dashed', linewidth=1)
        ax.axvline(lower_thresh_chan[channel_index] * np.std(image[:, :, channel_index]) + np.mean(image[:, :, channel_index]), color='g', linestyle='dashed', linewidth=1)

    # Hide x labels and tick labels for top plots and y ticks for right plots.
    for ax in axs.flat:
        ax.label_outer()

    plt.show()

    # Apply thresholds to find maximum values in the background-subtracted depending on channel of interest, avoiding very bright spots
    # Mask out very bright spots
    channel_thresh = 3 * np.std(background_subtracted_image[:, :, mask_channel]) + np.mean(background_subtracted_image[:, :, mask_channel])
    print(f'Channel threshold for mask channel {mask_channel+1}: {channel_thresh}')
    
    thresh = (background_subtracted_image[:, :, mask_channel] > channel_thresh) & (background_subtracted_image[:, :, mask_channel] < upper_thresh)

    # Label the thresholded regions and return the number of cells
    labels = measure.label(thresh)
    props = measure.regionprops(labels)

    # Create circular ROIs around detected points
    rois = []

    for prop in props:
        y, x = prop.centroid
        radius = radius_factor 
        rois.append((int(x), int(y), int(radius)))
    
    # Display circular ROIs on the background-subtracted masked channel
    roi_image = np.stack([normalize(background_subtracted_image[:, :, mask_channel])]*3, axis=-1)  # Convert to RGB

    for roi in rois:
        x, y, radius = roi
        rr, cc = draw.disk((y, x), radius, shape=roi_image.shape)
        roi_image[rr, cc] = [0, 1, 0]  # Green for ROIs

    plt.imshow(roi_image)
    plt.title(f'ROIs on Background-Subtracted Mask_Channel {mask_channel+1}')
    plt.show()

    print(f'Number of ROIs detected in Channel 1: {len(rois)}')

    # Calculate mean fluorescence values for each channel and check if the signal is present
    for channel_index in range(n_channels):
        channel = background_subtracted_image[:, :, channel_index]
        channel_thresh = lower_thresh_chan[channel_index] * np.std(channel) + np.mean(channel)

        for roi in rois:
            x, y, radius = roi
            rr, cc = draw.disk((y, x), radius, shape=channel.shape)
            roi_area = channel[rr, cc]
            mean_value = np.mean(roi_area)

            # Check if the signal is present and mark as positive or negative

            if mean_value > channel_thresh:
                positive_results[f'Channel {channel_index+1}'].append((x, y, radius))
                mean_fluorescence[f'Channel {channel_index+1}'].append(mean_value)
            else:
                positive_results[f'Channel {channel_index+1}'].append(None)
    
    fig, axs = plt.subplots(1, n_channels , figsize=(n_channels *5,20))

    adj_gain = 0.5
    axs[0].imshow((exposure.adjust_gamma(channel1,adj_gain)), cmap='Blues_r')
    axs[1].imshow((exposure.adjust_gamma(channel2,adj_gain)), cmap='Greens_r')
    axs[2].imshow((exposure.adjust_gamma(channel3,adj_gain)), cmap='Reds_r')
    if len(channels) == 4:
        axs[3].imshow((exposure.adjust_gamma(channel4,adj_gain)), cmap='Purples_r')

    for ax, channel_index in zip(axs, range(n_channels)):
        ax.set_title(f'Channel {channel_index+1} (Raw)')

    # Hide x labels and tick labels for top plots and y ticks for right plots.
    for ax in axs.flat:
        ax.label_outer()
        ax.axis('off')

    plt.show()

    # Display positive ROIs for each channel
    fig, ax = plt.subplots(1, n_channels, figsize=(n_channels * 5, 20))

    for channel_index in range(n_channels):
        channel_image = np.stack([normalize(background_subtracted_image[:, :, channel_index])]*3, axis=-1)  # Convert to RGB

        for roi in positive_results[f'Channel {channel_index+1}']:
            if roi is not None:
                x, y, radius = roi
                rr, cc = draw.disk((y, x), radius, shape=channel_image.shape)
                channel_image[rr, cc] = [0, 1, 0]  # Green for positive ROIs

        ax[channel_index].imshow(channel_image)
        ax[channel_index].set_title(f'Positive ROIs on Channel {channel_index + 1}')
        ax[channel_index].axis('off')
    
    plt.show()

    return mean_fluorescence, background_values, mean_background_value, positive_results

# Main Workflow Point

In [ ]:
# Alternative method to select the folder (open main folder with all subfolders)
main_folder_path = select_folder()

In [ ]:
# Save memory by closing all currently open figures
plt.close('all')

# # Open the folder which contains the images
# folder_path = select_folder()

# Process all sets of images in the folder and collect the results
results = []

# Thresholds for ROI detection and background exclusion
lower_thresh_factor = [1, 1.5, 1.5, 1.5]  # Adjust this value based on your needs
upper_thresh = 60000  # Adjust this value to exclude very bright spots
background_threshold = 0  # Adjust this value based on your needs
radius_factor = 15  # Factor to determine the radius of the circular ROIs
mask_channel = 1  # Channel to use for masking bright spots
channel_of_interest = 4  # Channel of interest for Background ROI detection

for root, dirs, files in os.walk(main_folder_path):
        for file in files:
            if file.lower().endswith('.ome'):
                # Use the .ome file name as the base name for the images
                base_name = os.path.splitext(file)[0].replace('.companion', '')
                ome_path = os.path.join(root, file)
                try:
                    # Find the associated channel images with variable suffixes (case-insensitive)
                    channel_files = [f for f in files if f.lower().startswith(base_name.lower()) and f.lower().endswith(('.tiff', '.tf2'))]
                    channel_files.sort(key=lambda f: f.lower())  # Sort files alphabetically ignoring case

                    channels = []
                    for channel_file in channel_files:
                        channel_path = os.path.join(root, channel_file)
                        channel_image = read_tiff_channel(channel_path)
                        channels.append(channel_image)
                    
                    mean_fluorescence, background_values, mean_background_value, positive_results = process_images(channels,
                                                                                                                lower_thresh_factor, 
                                                                                                                upper_thresh, 
                                                                                                                background_threshold, 
                                                                                                                radius_factor, 
                                                                                                                mask_channel, 
                                                                                                                channel_of_interest, 
                                                                                                                single_ch_background = True)
                except:
                    print(f"Error processing images for {base_name}")
                    pass
                    
                # Collect results into a list of dictionaries for easy conversion to DataFrame
                result = {'Base Name': base_name}
                for channel, values in mean_fluorescence.items():
                    result[f'{channel} Fluorescence mean value'] = np.mean(values)
                    # result[f'{channel} Background Values'] = background_values[channel]
                    result[f'{channel} Mean Background'] = np.mean(background_values[channel])
                    result[f'{channel} Positive Results'] = sum(x is not None for x in positive_results[channel]) 
                    result[f'{channel} Negative Results'] = sum(x is None for x in positive_results[channel])
                results.append(result)

# Convert the results to a DataFrame and save to CSV
df = pd.DataFrame(results)
df.to_csv(os.path.join(folder_path, 'mean_fluorescence_results.csv'), index=False)

print("Processing complete. Results saved to mean_fluorescence_results.csv")
